In [1]:
import pymongo
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["story_database"]  # Database name
collection = db["short_stories"]

documents = collection.find()

df_raw = pd.DataFrame(list(documents))
df_raw = df_raw[["text", "label"]]
df_raw.head(5)

,text,label
0,Transcriber's Note:\nEvery effort has been mad...,0
1,"“In a few moments Marianne, Solomin, Paul,\n\n...",0
2,“Marianne knelt beside the sofa.… Nezhdanof\n\...,0
3,"“Solomin raised Marianne's hand, her head\n\nl...",0
4,"“He now was no longer, but the hands of\n\nSol...",0


In [2]:
# Define the cleaning function
def clean_text(text):

    text = text.replace('\n', ' ') 
    text = re.sub(r'=', '', text)  

    # Keep only English alphabet characters and spaces 
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

df_raw['cleaned_text'] = df_raw['text'].apply(clean_text)

df = df_raw[["cleaned_text", "label"]]

# Vectorize the data


In [3]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

df_shuffled = df.sample(frac=1).reset_index(drop=True)


X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

cleaned_data = (classification_report(y_test, y_pred))
print(cleaned_data)

              precision    recall  f1-score   support

           0       0.97      0.91      0.94        32
           1       0.95      0.98      0.96        55

    accuracy                           0.95        87
   macro avg       0.96      0.94      0.95        87
weighted avg       0.95      0.95      0.95        87



Wow this was a bit anticlimactic, lets mess the data up see if we get the same results

# Lets not clean the data!

In [4]:
df_shuffled = df_raw.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.97      0.85      0.91        34
           1       0.91      0.98      0.95        53

    accuracy                           0.93        87
   macro avg       0.94      0.92      0.93        87
weighted avg       0.93      0.93      0.93        87



# Lets add data, fakenews!

In [5]:
fakenews = pd.read_csv("data/fake_news.csv")
fakenews.columns = ["cleaned_text", "label"]

In [6]:
data = pd.concat([fakenews, df], ignore_index=True)


In [7]:
df_shuffled = data.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.89      0.89      0.89        28
           1       0.82      0.98      0.89        51
           2       1.00      0.83      0.91        58

    accuracy                           0.90       137
   macro avg       0.90      0.90      0.90       137
weighted avg       0.91      0.90      0.90       137



In [8]:
confusion_matrix(y_test, y_pred)

array([[25,  3,  0],
       [ 1, 50,  0],
       [ 2,  8, 48]], dtype=int64)

## Is the vectorizer that good?

In [9]:
bbc_news = pd.read_csv("data/bbc_news.csv")
bbc = bbc_news[["cleaned_text", "category_encoded"]]
bbc.columns =  ["cleaned_text", "label"]

In [10]:
data = pd.concat([bbc, data], ignore_index=True)

In [11]:
data["label"].value_counts()

label
3    100
6    100
5    100
7    100
4    100
2    100
1    100
0     74
Name: count, dtype: int64

In [12]:
df_shuffled = data.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        25
           1       0.94      0.83      0.88        35
           2       0.96      0.76      0.85        33
           3       0.94      0.94      0.94        33
           4       0.81      0.97      0.88        31
           5       0.93      0.90      0.91        29
           6       0.78      0.95      0.86        22
           7       0.96      1.00      0.98        25

    accuracy                           0.91       233
   macro avg       0.91      0.92      0.91       233
weighted avg       0.92      0.91      0.91       233



In [13]:
confusion_matrix(y_test, y_pred)

array([[25,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 29,  0,  0,  2,  0,  4,  0],
       [ 0,  2, 25,  1,  2,  2,  0,  1],
       [ 0,  0,  1, 31,  1,  0,  0,  0],
       [ 0,  0,  0,  0, 30,  0,  1,  0],
       [ 0,  0,  0,  1,  1, 26,  1,  0],
       [ 0,  0,  0,  0,  1,  0, 21,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 25]], dtype=int64)

## Increasing the datas dimensions by adding more data as well as adding more things to classify is instead of worsening the model its getting improved?
The TfidfVectorizer proves to be an excellent tool for vectorizing text data, as it does a near-perfect job of capturing the essence of the information. However, it's important to note that a vectorizer like TfidfVectorizer thrives on large datasets. While adding more labels was part of the improvement, it was the accompanying increase in data that enabled the creation of a more complex sparse matrix, allowing the model to perform more efficiently and effectively.

## Lets create a transformer

In [14]:
df_deeplearning = df_shuffled

In [15]:
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Use tokenizer to encode
df_deeplearning['input_ids'] = df_deeplearning['cleaned_text'].apply(
    lambda x: tokenizer.encode(x, truncation=True, padding='max_length', max_length=128)
)

In [16]:
df_deeplearning

,cleaned_text,label,input_ids
0,"Connors rallying cry for British tennis ""Do y...",7,"[101, 6720, 2015, 8320, 2075, 5390, 2005, 2329..."
1,Angry constituents have inundated town hall me...,2,"[101, 4854, 24355, 2031, 1999, 18426, 3064, 22..."
2,Hillbillies singer Scoggins dies Country and ...,6,"[101, 2940, 24457, 3111, 3220, 8040, 8649, 165..."
3,Premier League planning Cole date The Premier...,7,"[101, 4239, 2223, 4041, 5624, 3058, 1996, 4239..."
4,Ruthenium hexafluoride also rutheniumVI fluori...,1,"[101, 7920, 18595, 2819, 2002, 18684, 10258, 1..."
...,...,...,...
769,This is a list of universities in Spain which ...,1,"[101, 2023, 2003, 1037, 2862, 1997, 5534, 1999..."
770,Singer Christina Aguilera to wed Pop star Chr...,6,"[101, 3220, 12657, 12943, 19231, 6906, 2000, 2..."
771,Teenager Tait picked for England Newcastles t...,7,"[101, 10563, 13843, 2102, 3856, 2005, 2563, 81..."
772,Mobile TV tipped as one to watch Scandinavian...,4,"[101, 4684, 2694, 11182, 2004, 2028, 2000, 342..."


In [17]:
input_ids = torch.tensor(df_deeplearning['input_ids'].tolist(), dtype=torch.long)
labels = torch.tensor(df_deeplearning['label'].tolist(), dtype=torch.float)  # Assuming multi-label

# Create torch dataset
dataset = TensorDataset(input_ids, labels)

# Create a DataLoader for batching and shuffling
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [18]:
class SimpleModel(nn.Module):
    def __init__(self, input_size, hidden_units, output_size):
        super(SimpleModel, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(input_size, hidden_units))
        self.layer2 = nn.Linear(hidden_units, output_size)


    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)  
        return x
    

model_0 = SimpleModel(input_size=128, hidden_units=16, output_size=8)

In [19]:
input_ids.size()

torch.Size([774, 128])

In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_0.parameters(), lr = 0.01)

In [21]:
def training_loop(loss_fn, optimizer, model,epochs=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):
        train_loss = 0
        for batch, (X, y) in enumerate(dataloader):
            model.train()
            

            X, y = X.to(device), y.to(device)

            X = X.float() 
            y = y.long()  
            
            y_pred = model_0(X)

            loss = loss_fn(y_pred, y)
            train_loss += loss.item()

            optimizer.zero_grad()

            loss.backward()
            optimizer.step()

        
        train_loss /= len(dataloader)
        
        if epoch % 20 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Average Training Loss: {train_loss:.4f}")


# Call the training loop
training_loop(loss_fn=loss_fn, optimizer=optimizer, model=model_0, epochs=100)


Epoch 1/100, Average Training Loss: 112.8398
Epoch 21/100, Average Training Loss: 2.0765
Epoch 41/100, Average Training Loss: 2.0753
Epoch 61/100, Average Training Loss: 2.0775
Epoch 81/100, Average Training Loss: 2.0766


In [22]:
class MoreAdvancedModel(nn.Module):
    def __init__(self, input_size, hidden_units, output_size):
        super().__init__()
        self.layer1 = nn.Sequential(nn.Linear(input_size, hidden_units))
        self.layer2 = nn.Linear(hidden_units, hidden_units)
        self.layer3 = nn.Linear(hidden_units, output_size)


    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)  
        return x
    

model_1 = MoreAdvancedModel(input_size=128, hidden_units=128, output_size=8)

In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_1.parameters(), lr = 0.01)

In [24]:
training_loop(loss_fn=loss_fn, optimizer=optimizer, model=model_1, epochs=100)

Epoch 1/100, Average Training Loss: 2.0724
Epoch 21/100, Average Training Loss: 2.0724
Epoch 41/100, Average Training Loss: 2.0724
Epoch 61/100, Average Training Loss: 2.0726
Epoch 81/100, Average Training Loss: 2.0726
